# Example of using OpenAI streaming with Guardrails


In [7]:
# Few imports and global variables
from rich import print
import guardrails as gd
import openai
from IPython.display import clear_output
import time

### 1. For structured JSON output


#### Define the prompt and output schema


In [8]:
from pydantic import BaseModel, Field
from typing import List
from guardrails.validators import (
    ValidRange,
    UpperCase,
    LowerCase,
    OneLine,
)

prompt = """
Given the following doctor's notes about a patient, please extract a dictionary that contains the patient's information.

${doctors_notes}

${gr.complete_json_suffix_v2}
"""

doctors_notes = """152 y/o female with chronic macular rash to face and hair, worse in beard, eyebrows and nares.
The rash is itchy, flaky and slightly scaly. Moderate response to OTC steroid cream. Patient has been using cream for 2 weeks and also suffers from diabetes."""


class Symptom(BaseModel):
    symptom: str = Field(description="Symptom that a patient is experiencing")
    affected_area: str = Field(
        description="What part of the body the symptom is affecting",
        validators=[
            LowerCase(on_fail="fix"),
        ],
    )


class Medication(BaseModel):
    medication: str = Field(
        description="Name of the medication the patient is taking",
        validators=[UpperCase(on_fail="fix")],
    )
    response: str = Field(description="How the patient is responding to the medication")


class PatientInfo(BaseModel):
    gender: str = Field(description="Patient's gender")
    age: int = Field(
        description="Patient's age",
        validators=[ValidRange(min=0, max=100, on_fail="fix")],
    )
    symptoms: List[Symptom] = Field(
        description="Symptoms that the patient is currently experiencing. Each symptom should be classified into  separate item in the list."
    )
    current_meds: List[Medication] = Field(
        description="Medications the patient is currently taking and their response"
    )
    miscellaneous: str = Field(
        description="Any other information that is relevant to the patient's health; something that doesn't fit into the other categories.",
        validators=[LowerCase(on_fail="fix"), OneLine(on_fail="fix")],
    )

#### Create the Guard object


In [9]:
guard = gd.Guard.from_pydantic(output_class=PatientInfo, prompt=prompt)

##### Example 1: No streaming

By default, the `stream` parameter is set to `False`


In [10]:
# Wrap the OpenAI API call with the `guard` object
raw_llm_output, validated_output, *rest = guard(
    openai.chat.completions.create,
    prompt_params={"doctors_notes": doctors_notes},
    max_tokens=1024,
    temperature=0.3,
)

# Print the validated output from the LLM
print(validated_output)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
    'gender': 'female',
    'age': 100,
    'symptoms': [
        {'symptom': 'chronic macular rash', 'affected_area': 'face'},
        {'symptom': 'itchy', 'affected_area': 'beard'},
        {'symptom': 'itchy', 'affected_area': 'eyebrows'},
        {'symptom': 'itchy', 'affected_area': 'nares'},
        {'symptom': 'flaky', 'affected_area': 'face'},
        {'symptom': 'flaky', 'affected_area': 'hair'},
        {'symptom': 'slightly scaly', 'affected_area': 'face'}
    ],
    'current_meds': [{'medication': 'OTC STEROID CREAM', 'response': 'moderate'}],
    'miscellaneous': 'patient also suffers from diabetes'
}

In [11]:
# Let's see the logs
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Given the following doctor's notes about a patient, please extract a dictionary that contains the       │ │
    │ │ patient's information.                                                                                  │ │
    │ │                                                                                                         │ │
    │ │ 152 y/o female with chronic macular rash to face and hair, worse in beard, eyebrows and nares.          │ │
    │ │ The rash is itchy, flaky and slightly scaly. Moderate response to OTC steroid cream. Patient has been   │ │
    │ │ using cream for 2 weeks and also suffers from diabetes.                                                 │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <string name="gender" description="Patient's gender"/>                                              │ │
    │ │     <integer name="age" description="Patient's age" format="valid-range: min=0 max=100"/>               │ │
    │ │     <list name="symptoms" description="Symptoms that the patient is currently experiencing. Each        │ │
    │ │ symptom should be classified into  separate item in the list.">                                         │ │
    │ │         <object>                                                                                        │ │
    │ │             <string name="symptom" description="Symptom that a patient is experiencing"/>               │ │
    │ │             <string name="affected_area" description="What part of the body the symptom is affecting"   │ │
    │ │ format="lower-case"/>                                                                                   │ │
    │ │         </object>                                                                                       │ │
    │ │     </list>                                                                                             │ │
    │ │     <list name="current_meds" description="Medications the patient is currently taking and their        │ │
    │ │ response">                                                                                              │ │
    │ │         <object>                                                                                        │ │
    │ │             <string name="medication" description="Name of the medication the patient is taking"        │ │
    │ │ format="upper-case"/>                                                                                   │ │
    │ │             <string name="response" description="How the patient is responding to the medication"/>     │ │
    │ │         </object>                                                                                       │ │
    │ │     </list>                                                                                             │ │
    │ │     <string name="miscellaneous" description="Any other information that is relevant to the patient's   │ │
    │ │ health; something that doesn't fit into the

##### Example 2: Streaming

Set the `stream` parameter to `True`


In [12]:
# Wrap the OpenAI API call with the `guard` object
fragment_generator = guard(
    openai.chat.completions.create,
    prompt_params={"doctors_notes": doctors_notes},
    max_tokens=1024,
    temperature=0,
    stream=True,
)


for op in fragment_generator:
    clear_output(wait=True)
    print(op)
    time.sleep(0.5)

Raw LLM response:
{
  "gender": "female",
  "age": 152,
  "symptoms": [
    {
      "symptom": "chronic macular rash",
      "affected_area": "face"
    },
    {
      "symptom": "chronic macular rash",
      "affected_area": "hair"
    },
    {
      "symptom": "chronic macular rash",
      "affected_area": "beard"
    },
    {
      "symptom": "chronic macular rash",
      "affected_area": "eyebrows"
    },
    {
      "symptom": "chronic macular rash",
      "affected_area": "nares"
    },
    {
      "symptom": "itchy",
      "affected_area": "rash"
    },
    {
      "symptom": "flaky",
      "affected_area": "rash"
    },
    {
      "symptom": "slightly scaly",
      "affected_area": "rash"
    }
  ],
  "current_meds": [
    {
      "medication": "OTC steroid cream",
      "response": "moderate"
    }
  ],
  "miscellaneous": "patient also suffers from diabetes"
}

Validated response:
{
    "gender": "female",
    "age": 100,
    "symptoms": [
        {
            "symptom": "chronic macular rash",
            "affected_area": "face"
        },
        {
            "symptom": "chronic macular rash",
            "affected_area": "hair"
        },
        {
            "symptom": "chronic macular rash",
            "affected_area": "beard"
        },
        {
            "symptom": "chronic macular rash",
            "affected_area": "eyebrows"
        },
        {
            "symptom": "chronic macular rash",
            "affected_area": "nares"
        },
        {
            "symptom": "itchy",
            "affected_area": "rash"
        },
        {
            "symptom": "flaky",
            "affected_area": "rash"
        },
        {
            "symptom": "slightly scaly",
            "affected_area": "rash"
        }
    ],
    "current_meds": [
        {
            "medication": "OTC STEROID CREAM",
            "response": "moderate"
        }
    ],
    "miscellaneous": "patient also suffers from diabetes"
}

In [13]:
# Let's see the logs
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Given the following doctor's notes about a patient, please extract a dictionary that contains the       │ │
    │ │ patient's information.                                                                                  │ │
    │ │                                                                                                         │ │
    │ │ 152 y/o female with chronic macular rash to face and hair, worse in beard, eyebrows and nares.          │ │
    │ │ The rash is itchy, flaky and slightly scaly. Moderate response to OTC steroid cream. Patient has been   │ │
    │ │ using cream for 2 weeks and also suffers from diabetes.                                                 │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <string name="gender" description="Patient's gender"/>                                              │ │
    │ │     <integer name="age" description="Patient's age" format="valid-range: min=0 max=100"/>               │ │
    │ │     <list name="symptoms" description="Symptoms that the patient is currently experiencing. Each        │ │
    │ │ symptom should be classified into  separate item in the list.">                                         │ │
    │ │         <object>                                                                                        │ │
    │ │             <string name="symptom" description="Symptom that a patient is experiencing"/>               │ │
    │ │             <string name="affected_area" description="What part of the body the symptom is affecting"   │ │
    │ │ format="lower-case"/>                                                                                   │ │
    │ │         </object>                                                                                       │ │
    │ │     </list>                                                                                             │ │
    │ │     <list name="current_meds" description="Medications the patient is currently taking and their        │ │
    │ │ response">                                                                                              │ │
    │ │         <object>                                                                                        │ │
    │ │             <string name="medication" description="Name of the medication the patient is taking"        │ │
    │ │ format="upper-case"/>                                                                                   │ │
    │ │             <string name="response" description="How the patient is responding to the medication"/>     │ │
    │ │         </object>                                                                                       │ │
    │ │     </list>                                                                                             │ │
    │ │     <string name="miscellaneous" description="Any other information that is relevant to the patient's   │ │
    │ │ health; something that doesn't fit into the

As you can see here, the outputs in both examples match. The only difference is that, in the streaming example, the outputs are returned as soon as they are received and validated by Guardrails. In the non-streaming example, the outputs are returned only after the entire request has been processed by the API. In other words, when streaming is enabled, the API returns the outputs as soon as they are ready, rather than waiting for the entire request to be processed.


### 2. For unstructured text output


#### Define the prompt and Guard object with validators


In [14]:
from guardrails.validators import UpperCase, OneLine

prompt = """
Generate a short description of large language models. Each new sentence should be on another line.
"""

guard = gd.Guard.from_string(
    validators=[
        UpperCase(on_fail="fix"),
        OneLine(on_fail="fix"),
    ],
    description="testmeout",
    prompt=prompt,
)

#### Example 1: No streaming

By default, the `stream` parameter is set to `False`


In [15]:
# Wrap the OpenAI API call with the `guard` object
raw, validated, *rest = guard(
    openai.chat.completions.create,
    max_tokens=50,
    temperature=0.1,
)

# Print the raw and validated outputs
print(f"Raw output:\n{raw}")
print(f"Validated output:\n{validated}")

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Raw output:
Large language models are powerful AI systems that can generate human-like text. They are trained on vast amounts 
of data, allowing them to understand and generate coherent sentences. These models have the ability to answer 
questions, write essays, and even create poetry. They

Validated output:
LARGE LANGUAGE MODELS ARE POWERFUL AI SYSTEMS THAT CAN GENERATE HUMAN-LIKE TEXT. THEY ARE TRAINED ON VAST AMOUNTS 
OF DATA, ALLOWING THEM TO UNDERSTAND AND GENERATE COHERENT SENTENCES. THESE MODELS HAVE THE ABILITY TO ANSWER 
QUESTIONS, WRITE ESSAYS, AND EVEN CREATE POETRY. THEY

#### Example 2: With streaming

Set the `stream` parameter to `True`


In [16]:
# Wrap the OpenAI API call with the `guard` object
fragment_generator = guard(
    openai.chat.completions.create,
    max_tokens=50,
    temperature=0.1,
    stream=True,
)


for op in fragment_generator:
    clear_output(wait=True)
    print(op)
    time.sleep(0.1)

Raw LLM response:
Large language models are powerful AI systems that can generate human-like text. They are trained on vast amounts 
of data, allowing them to understand and generate coherent sentences. These models have the ability to answer 
questions, write essays, and even create stories. They

Validated response:
"LARGE LANGUAGE MODELS ARE POWERFUL AI SYSTEMS THAT CAN GENERATE HUMAN-LIKE TEXT. THEY ARE TRAINED ON VAST AMOUNTS 
OF DATA, ALLOWING THEM TO UNDERSTAND AND GENERATE COHERENT SENTENCES. THESE MODELS HAVE THE ABILITY TO ANSWER 
QUESTIONS, WRITE ESSAYS, AND EVEN CREATE STORIES. THEY"

In [17]:
# See guard history
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Generate a short description of large language models. Each new sentence should be on another line.     │ │
    │ │                                                                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭───────────────────────────────────────────── Instructions ──────────────────────────────────────────────╮ │
    │ │ You are a helpful assistant, expressing yourself through a string.                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
    │ │ No message history.                                                                                     │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ Large language models are powerful AI systems that can generate human-like text. They are trained on    │ │
    │ │ vast amounts of data, allowing them to understand and generate coherent sentences. These models have    │ │
    │ │ the ability to answer questions, write essays, and even create stories. They                            │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ 'LARGE LANGUAGE MODELS ARE POWERFUL AI SYSTEMS THAT CAN GENERATE HUMAN-LIKE TEXT. THEY ARE TRAINED ON   │ │
    │ │ VAST AMOUNTS OF DATA, ALLOWING THEM TO UNDERSTAND AND GENERATE COHERENT SENTENCES. THESE MODELS HAVE    │ │
    │ │ THE ABILITY TO ANSWER QUESTIONS, WRITE ESSAYS, AND EVEN CREATE STORIES. THEY'                           │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

As you can see, the outputs in both examples match. The only difference is that, in the streaming example, the outputs are returned as soon as they are received and validated by Guardrails. In the non-streaming example, the outputs are returned only after the entire request has been processed by the API. In other words, when streaming is enabled, the API returns the outputs as soon as they are ready, rather than waiting for the entire request to be processed.

This also works with openai's chat completion API.
